In [134]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers


import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OneHotEncoder



from plotnine import *

from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score

from sklearn.linear_model import LogisticRegression # Linear Regression Model
from sklearn.preprocessing import StandardScaler #Z-score variables
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error


from sklearn.compose import make_column_transformer


from sklearn.model_selection import train_test_split # simple TT split cv
from sklearn.model_selection import KFold # k-fold cv


Choose a dataset that you're interested in from among these options (or choose your own data
set as long as it's large enough and you check with me in advance)

Apple Quality: https://www.kaggle.com/datasets/nelgiriyewithana/apple-quality

In [135]:
apple_data = pd.read_csv("apple_quality.csv")
apple_data

,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,0.0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590483,good
1,1.0,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809367,good
2,2.0,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636473,bad
3,3.0,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723217,good
4,4.0,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984036,good
...,...,...,...,...,...,...,...,...,...
3996,3996.0,-0.293118,1.949253,-0.204020,-0.640196,0.024523,-1.087900,1.854235285,good
3997,3997.0,-2.634515,-2.138247,-2.440461,0.657223,2.199709,4.763859,-1.334611391,bad
3998,3998.0,-4.008004,-1.779337,2.366397,-0.200329,2.161435,0.214488,-2.229719806,good
3999,3999.0,0.278540,-1.715505,0.121217,-1.154075,1.266677,-0.776571,1.599796456,good


In [136]:
'''
you've clean your data (e.g. one hot encoded, got rid of dollar signs...etc) as
needed before training
'''

# drop missing values
missing_values = apple_data.isnull().sum()
apple_data.dropna(inplace = True)


# quality is cateogorical (good, bad) encoded to (1, 0)
label_encoder = LabelEncoder()
apple_data['Quality'] = label_encoder.fit_transform(apple_data['Quality'])

In [137]:
# FEET
feet = ["Size", "Weight", "Sweetness", "Crunchiness", "Juiciness", "Ripeness", "Acidity"]
predictors = ["Quality"]

X = apple_data[feet]
y = apple_data[predictors]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



''' you've properly z-scored or otherwise scaled your data before training '''
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

1. Deep FEED FORWARD Neural Network

In [138]:
'''your **model architechture** and **loss function** are appropriate for the problem'''
# Build the model for binary classification
model = Sequential([
    Dense(250, input_shape=[7], kernel_regularizer=regularizers.l2(0.001), activation='relu'),
    Dropout(0.5),  # Dropout to prevent overfitting
    Dense(100, kernel_regularizer=regularizers.l2(0.001), activation='relu'),
    Dropout(0.5),
    Dense(50, kernel_regularizer=regularizers.l2(0.001), activation='relu'),
    Dense(30, kernel_regularizer=regularizers.l2(0.001), activation='relu'),
    Dense(20, kernel_regularizer=regularizers.l2(0.001), activation='relu'),
    Dense(10, kernel_regularizer=regularizers.l2(0.001), activation='relu'),
    Dense(1, activation='sigmoid')  # Single output unit for binary classification
])

model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=0.001),  # Adam optimizer with lower learning rate
    metrics=['mae', 'mse', 'accuracy']
)


history = model.fit(
    X_train_scaled, y_train,
    epochs=50,
    batch_size=16,
    validation_data=(X_test_scaled, y_test),
)

train_loss, train_mae, train_mse, train_acc = model.evaluate(X_train_scaled, y_train, verbose=0)
test_loss, test_mae, test_mse, test_acc = model.evaluate(X_test_scaled, y_test, verbose=0)

'''you print out **at least 2 metrics** for both train and test data to examine'''
print(f"Training Metrics: Accuracy: {train_acc:.4f}, MAE: {train_mae:.4f}, MSE: {train_mse:.4f}, Loss: {train_loss:.4f}")
print(f"Testing Metrics: Accuracy: {test_acc:.4f}, MAE: {test_mae:.4f}, MSE: {test_mse:.4f}, Loss: {test_loss:.4f}")

Epoch 1/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.6255 - loss: 0.8812 - mae: 0.4390 - mse: 0.2165 - val_accuracy: 0.8675 - val_loss: 0.5279 - val_mae: 0.2054 - val_mse: 0.0960
Epoch 2/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.8262 - loss: 0.5936 - mae: 0.2481 - mse: 0.1244 - val_accuracy: 0.8763 - val_loss: 0.4768 - val_mae: 0.1857 - val_mse: 0.0902
Epoch 3/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8504 - loss: 0.5037 - mae: 0.2143 - mse: 0.1064 - val_accuracy: 0.8863 - val_loss: 0.4507 - val_mae: 0.2126 - val_mse: 0.0893
Epoch 4/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8527 - loss: 0.4737 - mae: 0.2159 - mse: 0.1050 - val_accuracy: 0.8888 - val_loss: 0.4148 - val_mae: 0.1833 - val_mse: 0.0832
Epoch 5/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8569 - loss: 0.4513 - mae: 0.2076 - mse: 0.1019 - val_accuracy: 0.8975 - val_loss: 0.3937 - val_mae: 0.1756 - val_mse: 0.0807
Epoch 6/50
200/200 ━━━━━━━━━━━━━━━

2. Linear Regression

In [ ]:
log_reg_model = LogisticRegression()
log_reg_model.fit(X_train_scaled, y_train)

# Predict on training and test sets
y_train_pred = log_reg_model.predict(X_train_scaled)
y_test_pred = log_reg_model.predict(X_test_scaled)

# Calculate performance metrics
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

# Print out the metrics
print(f"Logistic Regression Training Metrics: Accuracy: {train_accuracy:.4f}, MAE: {train_mae:.4f}, MSE: {train_mse:.4f}")
print(f"Logistic Regression Testing Metrics: Accuracy: {test_accuracy:.4f}, MAE: {test_mae:.4f}, MSE: {test_mse:.4f}")

Logistic Regression Training Metrics: Accuracy: 0.7453, MAE: 0.2547, MSE: 0.2547
Logistic Regression Testing Metrics: Accuracy: 0.7538, MAE: 0.2462, MSE: 0.2462
